In [11]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import * 
from pyspark.sql import functions as F
from pyspark.sql.functions import max

spark = SparkSession.Builder().appName('intro_spark')\
        .config("spark.sql.session.logLevel","WARN")\
        .config('spark.executor.instances',"3")\
        .config('spark.executor.cores',"4")\
        .config('spark.executor.memory',"4g")\
        .config('spark.driver.cores',"2")\
        .config('spark.driver.memory',"1g")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark.conf.set('spark.sql.suffle.partitions','5')

In [2]:
%%time
flightData2010 = spark\
                .read\
                .option('inferSchema','true')\
                .option('header','true')\
                .csv("Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv")
flightData2010.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [5]:
 flightData2010.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='Equatorial Guinea', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1)]

In [6]:
flightData2010.createOrReplaceTempView('flightData2010')

In [7]:
sqlWay = spark.sql(
    """
    SELECT DEST_COUNTRY_NAME, COUNT(1) 
    FROM flightData2010
    GROUP BY DEST_COUNTRY_NAME
    """
)

dfWay = flightData2010.groupBy('DEST_COUNTRY_NAME').count()

In [8]:
sqlWay.explain()
dfWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 200), ENSURE_REQUIREMENTS, [plan_id=51]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/de-ninja/Documents/Courses/Learn-Spark-definitive-guide/Spa..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 200), ENSURE_REQUIREMENTS, [plan_id=64]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: In

In [10]:
sqlWay.show()
dfWay.show()

+--------------------+--------+
|   DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
|            Anguilla|       1|
|              Russia|       1|
|            Paraguay|       1|
|             Senegal|       1|
|              Sweden|       1|
|            Kiribati|       1|
|              Guyana|       1|
|         Philippines|       1|
|            Malaysia|       1|
|           Singapore|       1|
|                Fiji|       1|
|              Turkey|       1|
|             Germany|       1|
|         Afghanistan|       1|
|              Jordan|       1|
|               Palau|       1|
|Turks and Caicos ...|       1|
|              France|       1|
|              Greece|       1|
|              Taiwan|       1|
+--------------------+--------+
only showing top 20 rows

+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|            Anguilla|    1|
|              Russia|    1|
|            Paraguay|    1|
|             Senegal|    1|
|  

In [15]:
flightData2010.select(max('count')).take(1)

[Row(max(count)=348113)]

##### Q. Get the top 5 dest country with highest number of flights

In [23]:
spark.sql(
    """
    SELECT DEST_COUNTRY_NAME, SUM(count) AS num_of_flights
    FROM flightData2010 
    GROUP BY DEST_COUNTRY_NAME 
    ORDER BY 2 DESC
    LIMIT 5
    """
).show()

+-----------------+--------------+
|DEST_COUNTRY_NAME|num_of_flights|
+-----------------+--------------+
|    United States|        384932|
|           Canada|          8271|
|           Mexico|          6200|
|   United Kingdom|          1629|
|          Germany|          1392|
+-----------------+--------------+



In [38]:
flightData2010.groupBy('DEST_COUNTRY_NAME')\
.sum('count')\
.withColumnRenamed('sum(count)','destination_total')\
.orderBy(F.desc('destination_total'))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           384932|
|           Canada|             8271|
|           Mexico|             6200|
|   United Kingdom|             1629|
|          Germany|             1392|
+-----------------+-----------------+

